# Colab Setup and Data EDA

Mount Google Drive, set up the environment, and perform quick EDA for datasets in `My Drive/datasets`.

In [ ]:
# If running in Colab, mount Drive
try:
    from google.colab import drive  # type: ignore
    drive.mount('/content/drive')
    IN_COLAB = True
except Exception:
    IN_COLAB = False
print('IN_COLAB =', IN_COLAB)

In [ ]:
# Install deps if needed (Colab)
if 'google.colab' in str(get_ipython()):
    repo_url = 'https://github.com/Thabhelo/3d_medical_segemntation.git'
    drive_root = '/content/drive/MyDrive'
    repo_dir = f'{drive_root}/3d_medical_segmentation'
    import os, sys, subprocess
    if not os.path.exists(repo_dir):
        subprocess.run(['git', 'clone', repo_url, repo_dir], check=True)
    if repo_dir not in sys.path:
        sys.path.append(repo_dir)
    subprocess.run(['python', '-m', 'pip', 'install', '-q', '-U', 'pip'], check=True)
    subprocess.run(['python', '-m', 'pip', 'install', '-q', '-r', f'{repo_dir}/requirements.txt'], check=True)
print('Dependencies prepared.')

In [ ]:
# Set paths
from pathlib import Path
import sys
sys.path.append('/content/drive/MyDrive/3d_medical_segmentation') if IN_COLAB else None
from src.utils.io import get_drive_root, get_datasets_root, get_project_root

drive_root = get_drive_root()
data_root = get_datasets_root()
project_root = get_project_root()
print('data_root:', data_root)
print('project_root:', project_root)

In [ ]:
# Quick EDA: list datasets and sample files
import os
name_synonyms = {
    'BraTS': ['BraTS', 'brats', 'BRATS', 'BraTS2021'],
    'MSD': ['MSD', 'MSD_Liver', 'Task03_Liver'],
    'TotalSegmentator': ['TotalSegmentator', 'TotalSegmentor', 'TotalSeg', 'TotalSegmantator', 'totalseg']
}
for canonical, options in name_synonyms.items():
    p = None
    for opt in options:
        candidate = data_root / opt
        if candidate.exists():
            p = candidate
            break
    print(f"\nDataset: {canonical} ->", p if p else (data_root / options[0]))
    if p and p.exists():
        for root, dirs, files in os.walk(p):
            print(' ', root)
            for f in files[:5]:
                print('   -', f)
            break
    else:
        print('  MISSING - Please sync to Google Drive.')
